# Setting initial variables

## Importing libraries

In [1]:
import yaml
import numpy as np
import pandas as pd
from constants import DURATION_BX, VDRIFT
from mappings import Mapping
from timeit import timeit
import tracemalloc

## Reading raw data

In [2]:
CONFIG_FNAME = "../config/RUN000054.yml"
DATA_FNAME = "../data/RUN000054_data100.txt"

with open(CONFIG_FNAME, "r") as f:
        cfg = yaml.safe_load(f)

dtype_dict = { 'HEAD':np.uint8, 'FPGA':np.uint8, 'TDC_CHANNEL':np.uint8, 'ORBIT_CNT':np.uint64, 'BX_COUNTER':np.uint16, 'TDC_MEAS':np.uint8 }
stream_df = pd.read_csv(DATA_FNAME, dtype=dtype_dict)

# Optimization comparisons

## Mappings.py

### Raw code

#### Optimized

In [3]:
from constants import XCELL, X_POS_SHIFT, Z_POS_SHIFT

class Mapping_o(object):
    def __init__(self, cfg):
        self.cfg = cfg

    def sl_map(self, df):
        """
        Add columns with SL, LAYER, WIRE_NUM, WIRE_POS
        for local coordinates
        Args:
            - df  : Pandas dataframe of hits
            - cfg : configuration dict for this run
        """
        #print("local mapping...")
        # assing SL to each hit
        # sl_cfg is similar to {"fpga": 0, "ch_start": 0, "ch_end": 63}
        for sl, sl_cfg in self.cfg["sl_mapping"].items():
            sl_mask = (
                (df["FPGA"] == sl_cfg["fpga"])
                & (df["TDC_CHANNEL"] >= sl_cfg["ch_start"])
                & (df["TDC_CHANNEL"] <= sl_cfg["ch_end"])
            )
            df.loc[sl_mask, "SL"] = sl
        
        # create the layer column (layer 4 is the top one)
        df.loc[(df["TDC_CHANNEL"] % 4 == 0), "LAYER"] = 4
        df.loc[(df["TDC_CHANNEL"] % 4 == 2), "LAYER"] = 3
        df.loc[(df["TDC_CHANNEL"] % 4 == 1), "LAYER"] = 2
        df.loc[(df["TDC_CHANNEL"] % 4 == 3), "LAYER"] = 1

        df = df.astype({"SL": "uint8", "LAYER": "uint8"})
        # set the wire num inside the layer: ranging from 1 to 16 (left to right)
        # tdc_channel is normalized from 0->63 for each sl
        # assign the wire position
        for layer in [1, 2, 3, 4]:
            # local wire x position
            df.loc[df["LAYER"] == layer, "WIRE_X_LOC"] = (
                df["TDC_CHANNEL"] % 64 // 4
            ) * XCELL + X_POS_SHIFT[layer]

            # local wire z position
            df.loc[df["LAYER"] == layer, "WIRE_Z_LOC"] = Z_POS_SHIFT[layer]

        return df

    def global_map(self, df):
        """
        Create global coordinates based on the SL geometry
        adopted in the selected run
        Args:
            - df  : Pandas dataframe of hits
            - cfg : configuration dict for this run
        """
        # build the map for each sl
        df = self.sl_map(df)
        #print("global mapping...")
        # place wire in the space
        for sl, sl_shift in self.cfg["sl_shift"].items():
            # shift z
            df.loc[df["SL"] == sl, "WIRE_Z_GLOB"] = df["WIRE_Z_LOC"] + sl_shift["z"]

            # shift x
            df.loc[df["SL"] == sl, "WIRE_X_GLOB"] = df["WIRE_X_LOC"] + sl_shift["x"]

            # shift y -> not implemented

        return df
    
    
def prepare_for_mapping(stream_df):
    #drop NaN
    stream_df = stream_df.dropna()

    # create a dataframe with only valid hits ->
    # trigger words and scintillator hits are removed
    hits_df = stream_df[
        (stream_df.HEAD == cfg["headers"]["valid_hit"]) &
        (stream_df.TDC_CHANNEL <= 127)
        ]
     
    # select all orbits with a trigger signal from
    # the scintillators coincidence
    trigger_df = stream_df[
        (stream_df["HEAD"] == cfg["scintillator"]["head"]) & 
        (stream_df["FPGA"] == cfg["scintillator"]["fpga"]) & 
        (stream_df["TDC_CHANNEL"] == cfg["scintillator"]["tdc_ch"])
    ]

    del stream_df

    # create a T0 column (in ns)
    trigger_df["T0_NS"] = (trigger_df["BX_COUNTER"] + trigger_df["TDC_MEAS"] / 30) * DURATION_BX

    # select only hits in the same orbit of a scint trigger signal
    hits_df_ = pd.merge(
        hits_df, trigger_df[["ORBIT_CNT","T0_NS"]],
        left_on="ORBIT_CNT", right_on="ORBIT_CNT",
        suffixes=(None, None)
    )

    del trigger_df
    del hits_df
    return hits_df_


#### Not Optimized

In [4]:
from constants import XCELL, X_POS_SHIFT, Z_POS_SHIFT

class Mapping_no(object):
    def __init__(self, cfg):
        self.cfg = cfg

    def sl_map(self, df):
        """
        Add columns with SL, LAYER, WIRE_NUM, WIRE_POS
        for local coordinates
        Args:
            - df  : Pandas dataframe of hits
            - cfg : configuration dict for this run
        """

        # assing SL to each hit
        # sl_cfg is similar to {"fpga": 0, "ch_start": 0, "ch_end": 63}
        for sl, sl_cfg in self.cfg["sl_mapping"].items():
            sl_mask = (
                (df["FPGA"] == sl_cfg["fpga"])
                & (df["TDC_CHANNEL"] >= sl_cfg["ch_start"])
                & (df["TDC_CHANNEL"] <= sl_cfg["ch_end"])
            )
            df.loc[sl_mask, "SL"] = sl

        # create the layer column (layer 4 is the top one)
        df.loc[(df["TDC_CHANNEL"] % 4 == 0), "LAYER"] = 4
        df.loc[(df["TDC_CHANNEL"] % 4 == 2), "LAYER"] = 3
        df.loc[(df["TDC_CHANNEL"] % 4 == 1), "LAYER"] = 2
        df.loc[(df["TDC_CHANNEL"] % 4 == 3), "LAYER"] = 1

        # set the wire num inside the layer: ranging from 1 to 16 (left to right)
        # in tdc_channel_norm the channel is normalized from 0->63 for each sl
        df["TDC_CHANNEL_NORM"] = df["TDC_CHANNEL"] % 64  # .astype(np.uint8)
        df["WIRE_NUM"] = df["TDC_CHANNEL_NORM"] // 4 + 1  # .astype(np.uint8)

        # assign the wire position
        for layer in [1, 2, 3, 4]:
            # local wire x position
            df.loc[df["LAYER"] == layer, "WIRE_X_LOC"] = (
                df["WIRE_NUM"] - 1
            ) * XCELL + X_POS_SHIFT[layer]

            # local wire z position
            df.loc[df["LAYER"] == layer, "WIRE_Z_LOC"] = Z_POS_SHIFT[layer]

        df = df.astype({"SL": "int8", "LAYER": "int8"})
        return df

    def global_map(self, df):
        """
        Create global coordinates based on the SL geometry
        adopted in the selected run
        Args:
            - df  : Pandas dataframe of hits
            - cfg : configuration dict for this run
        """
        # build the map for each sl
        df = self.sl_map(df)

        # place wire in the space
        for sl, sl_shift in self.cfg["sl_shift"].items():
            # shift z
            df.loc[df["SL"] == sl, "WIRE_Z_GLOB"] = df["WIRE_Z_LOC"] + sl_shift["z"]

            # shift x
            df.loc[df["SL"] == sl, "WIRE_X_GLOB"] = df["WIRE_X_LOC"] + sl_shift["x"]

            # shift y -> not implemented

        return df


### Comparison

In [5]:
n_iter    = 5
mapper_o  = Mapping_o(cfg)
mapper_no = Mapping_no(cfg)
map_df_o  = prepare_for_mapping(stream_df)
map_df_no = prepare_for_mapping(stream_df)

tracemalloc.start()
t_o        = timeit('mapper_o.global_map(map_df_o)', number = n_iter, globals=globals()) / n_iter
_, peak_o  = tracemalloc.get_traced_memory()

tracemalloc.reset_peak()

t_no       = timeit('mapper_no.global_map(map_df_no)', number = n_iter, globals=globals()) / n_iter
_, peak_no = tracemalloc.get_traced_memory()

tracemalloc.stop()

print("Not optimized:\nTime {0} s, Memory {1} MB\nOptimized:\nTime {2} s, Memory {3} MB".format(t_no, peak_no/10**6, t_o, peak_o/10**6) )

Not optimized:
Time 0.07586992280121194 s, Memory 12.858104 MB
Optimized:
Time 0.13954538560064975 s, Memory 8.709441 MB


## EventsFactory.py
### Reading df from file comparison

In [6]:
dtype_dict = { 'HEAD':np.uint8, 'FPGA':np.uint8, 'TDC_CHANNEL':np.uint8, 'ORBIT_CNT':np.uint64, 'BX_COUNTER':np.uint16, 'TDC_MEAS':np.uint8 }
n_iter     = 5

tracemalloc.start()
tr_o        = timeit('pd.read_csv(DATA_FNAME, dtype=dtype_dict)', number = n_iter, globals=globals()) / n_iter
_, peak_o  = tracemalloc.get_traced_memory()

tracemalloc.reset_peak()

tr_no       = timeit('pd.read_csv(DATA_FNAME)', number = n_iter, globals=globals()) / n_iter
_, peak_no = tracemalloc.get_traced_memory()

tracemalloc.stop()

print("Not optimized:\nTime {0} s, Memory {1} MB\nOptimized:\nTime {2} s, Memory {3} MB".format(tr_no, peak_no/10**6, tr_o, peak_o/10**6) )

Not optimized:
Time 1.3509621144010453 s, Memory 326.82871 MB
Optimized:
Time 1.3727195213999948 s, Memory 122.58949 MB


### GetEvents

#### Raw Code

##### Optimized

In [7]:
from constants import DURATION_BX, VDRIFT


def map_hit_position(hits_df_, local=False):
    """
    Compute the position of the hit.
    If local=True the positions left/right will 
    be computed in the local frame of reference
    (e.g. X_LEFT_LOC). Else, the positions will be
    computed in the global frame of reference
    """

    hits_df_["HIT_DRIFT_TIME"] = (hits_df_["BX_COUNTER"]+hits_df_["TDC_MEAS"]/30)*DURATION_BX-hits_df_["T0_NS"]

    ref = "LOC" if local else "GLOB"
    hits_df_[f"X_LEFT_{ref}"] = hits_df_[f"WIRE_X_{ref}"] - hits_df_["HIT_DRIFT_TIME"]*VDRIFT
    hits_df_[f"X_RIGHT_{ref}"] = hits_df_[f"WIRE_X_{ref}"] + hits_df_["HIT_DRIFT_TIME"]*VDRIFT

    return hits_df_


def computeEvents_o(hits_df_):
    events = [group for _, group in hits_df_.groupby("ORBIT_CNT") if len(group) <= 32]
    return events


def getEvents_o(df_fname, cfg, runTimeShift):
    
    #reading df from file
    dtype_dict = { 'HEAD':np.uint8, 'FPGA':np.uint8, 'TDC_CHANNEL':np.uint8, 'ORBIT_CNT':np.uint64, 'BX_COUNTER':np.uint16, 'TDC_MEAS':np.uint8 }
    #print("Reading dataset from file...")
    stream_df = pd.read_csv(df_fname, dtype=dtype_dict)
    #drop NaN
    stream_df.dropna(inplace=True)

    # create a dataframe with only valid hits ->
    # trigger words and scintillator hits are removed
    hits_df = stream_df[
        (stream_df.HEAD == cfg["headers"]["valid_hit"]) &
        (stream_df.TDC_CHANNEL <= 127)
        ]
     
    # select all orbits with a trigger signal from
    # the scintillators coincidence
    trigger_df = stream_df[
        (stream_df["HEAD"] == cfg["scintillator"]["head"]) & 
        (stream_df["FPGA"] == cfg["scintillator"]["fpga"]) & 
        (stream_df["TDC_CHANNEL"] == cfg["scintillator"]["tdc_ch"])
    ]

    del stream_df

    # create a T0 column (in ns)
    trigger_df["T0_NS"] = (trigger_df["BX_COUNTER"] + trigger_df["TDC_MEAS"] / 30) * DURATION_BX

    # select only hits in the same orbit of a scint trigger signal
    hits_df_ = pd.merge(
        hits_df, trigger_df[["ORBIT_CNT","T0_NS"]],
        left_on="ORBIT_CNT", right_on="ORBIT_CNT",
        suffixes=(None, None)
    )

    del trigger_df
    del hits_df
    # print the number of valid hits found in data
    print(f"Valid hits: {hits_df_.shape[0]}")

    # create mapping with the loaded configurations
    mapper = Mapping_o(cfg)
    # map hits
    hits_df_ = mapper.global_map(hits_df_)

    # TIME SHIFTING
    # apply scintillator calibration -> they shoul be computed
    # for each run! For this example run, values are provided in
    # the configurations file.

    for sl, offset_sl in cfg["time_offset_sl"].items():
        # correction is in the form:
        # coarse offset valid for all SL + fine tuning 
        hits_df_.loc[
            hits_df_["SL"] == sl, "T0_NS"
        ] -= cfg["time_offset_scint"] -runTimeShift + offset_sl # 6 for 123, 8 for 0

    # having the time allows us to compute hit position 
    # with left / right ambiguity

    hits_df_ = map_hit_position(hits_df_, local=False)

    return computeEvents_o(hits_df_)

##### Not Optimized

In [8]:
def computeEvents_no(hits_df_):
    events = []
    for x in pd.unique(hits_df_.ORBIT_CNT):
        events.append(hits_df_[hits_df_["ORBIT_CNT"] == x])
        events[-1] = events[-1].reset_index(drop=True)
    return events


def getEvents_no(stream_df, cfg, runTimeShift):
    
    # create a dataframe with only valid hits ->
    # trigger words and scintillator hits are removed
    hits_df = stream_df[
        (stream_df.HEAD == cfg["headers"]["valid_hit"]) &
        (stream_df.TDC_CHANNEL <= 127)
        ].copy()

    # fix TDC_MEAS data type
    hits_df = hits_df.dropna()
    hits_df = hits_df.astype({"TDC_MEAS": "int32"})

    # create mapping with the loaded configurations
    mapper = Mapping_no(cfg)

    # map hits
    hits_df = mapper.global_map(hits_df)

    # select all orbits with a trigger signal from
    # the scintillators coincidence
    trigger_df = stream_df[
        (stream_df["HEAD"] == cfg["scintillator"]["head"]) & 
        (stream_df["FPGA"] == cfg["scintillator"]["fpga"]) & 
        (stream_df["TDC_CHANNEL"] == cfg["scintillator"]["tdc_ch"])
    ].copy()

    # create a T0 column (in ns)
    trigger_df["T0"] = (trigger_df["BX_COUNTER"] + trigger_df["TDC_MEAS"] / 30)

    # select only hits in the same orbit of a scint trigger signal
    hits_df_ = pd.merge(
        hits_df, trigger_df[["ORBIT_CNT","T0"]],
        left_on="ORBIT_CNT", right_on="ORBIT_CNT",
        suffixes=(None, None)
    )
    # print the number of valid hits found in data
    print(f"Valid hits: {hits_df_.shape[0]}")

    # TIME SHIFTING
    # apply scintillator calibration -> they shoul be computed
    # for each run! For this example run, values are provided in
    # the configurations file.

    # create a time column in NS
    hits_df_["T0_NS"] = hits_df_["T0"] * DURATION_BX

    for sl, offset_sl in cfg["time_offset_sl"].items():
        # correction is in the form:
        # coarse offset valid for all SL + fine tuning 
        hits_df_.loc[
            hits_df_["SL"] == sl, "T0_NS"
        ] -= cfg["time_offset_scint"] -runTimeShift + offset_sl # 6 for 123, 8 for 0

    # having the time allows us to compute hit position 
    # with left / right ambiguity

    hits_df_ = map_hit_position(hits_df_, local=False)

    return computeEvents_no(hits_df_)

#### Comparison

In [9]:
n_iter    = 1
stream_df = pd.read_csv(DATA_FNAME, dtype=dtype_dict)

tracemalloc.start()

t_no       = timeit('getEvents_no(stream_df, cfg, 0)', number = n_iter, globals=globals()) / n_iter
_, peak_no = tracemalloc.get_traced_memory()

tracemalloc.reset_peak()

t_o       = timeit('getEvents_o(DATA_FNAME, cfg, 0)', number = n_iter, globals=globals()) / n_iter - tr_o
_, peak_o = tracemalloc.get_traced_memory()

tracemalloc.stop()

print("Not optimized:\nTime {0} s, Memory {1} MB\nOptimized:\nTime {2} s, Memory {3} MB".format(t_no, peak_no/10**6, t_o, peak_o/10**6) )

Valid hits: 54952
Valid hits: 54952
Not optimized:
Time 7.853843677003169 s, Memory 689.836867 MB
Optimized:
Time 0.7442496255986044 s, Memory 252.461793 MB


# Reco.py

## Compute()

### Raw Code

#### Optimized

In [10]:
from pandas.core.indexes.numeric import IntegerIndex
from scipy import optimize
from numpy import sqrt


# *****************************************************************************
# LINEAR_REG
# *****************************************************************************


def chisq(X, Y, SY, a, b):
    return sum(((y - a - x * b) / sy) ** 2 for x, y, sy in zip(X, Y, SY))


def fitfunc(x, a, b):
    return a + b * x


def linear_reg(X, Y):

    sigma_X = [0.4] * len(X)  # 400 um std
    mGuess = 100 if ((X[0] - X[1]) == 0) else (Y[0] - Y[1]) / (X[0] - X[1])
    qGuess = Y[0] - mGuess * X[0]
    p_init = [qGuess, mGuess]  # valori iniziali
    p_best, pcov = optimize.curve_fit(fitfunc, Y, X, sigma=sigma_X, p0=p_init)

    chi2 = chisq(Y, X, sigma_X, p_best[0], p_best[1])
    dof = len(X) - 2  # - 1
    chisq_comp = abs(chi2 - dof) / sqrt(2 * dof)

    m = p_best[1]
    q = p_best[0]
    return {"m": m, "q": q, "chisq_comp": chisq_comp}


from itertools import combinations


def compute_o(df): 
    
    comb = []
    if len(df.LAYER.unique()) == 3:
        comb.append(df)
        tot_Hits = 3
    else:
        for index in list(combinations(df.index, 4)):
            if len(df.loc[index, :].LAYER.unique()) == 4:
                comb.append(df.loc[index, :]) 
        tot_Hits = 4

    min_lambda = np.finfo(float).max

    for data in comb:
        X = np.array(pd.concat([data["X_RIGHT_GLOB"], data["X_LEFT_GLOB"]]))
        Y = np.array(pd.concat([data["WIRE_Z_GLOB"], data["WIRE_Z_GLOB"]]))
        for indexes_comb in list(combinations(range(len(X)), tot_Hits)):
            indexes_comb = list(indexes_comb)
            if len(np.unique(Y[indexes_comb])) == tot_Hits:
                regr_dict = linear_reg(X[indexes_comb], Y[indexes_comb])
                if abs(regr_dict["chisq_comp"]) < min_lambda:
                    min_lambda = abs(regr_dict["chisq_comp"])
                    xdata = X[indexes_comb]
                    res_dict = regr_dict
                    best_comb = indexes_comb
                    best_data = data

    reco_df = pd.concat([best_data, best_data], axis=0, ignore_index=True)
    reco_df = reco_df.loc[best_comb, :]
    reco_df["m"] = np.full(len(reco_df), res_dict["m"])
    reco_df["q"] = np.full(len(reco_df), res_dict["q"])
    reco_df["X"] = xdata
    if xdata is None: return

    return reco_df

#### Not Optimized

In [11]:
def compute_no(df):
    
    comb = []
    if len(df.LAYER.unique()) == 3:
        comb.append(df)
        tot_Hits = 3
    else:
        for index in list(combinations(df.index, 4)):
            tmp_df = df.loc[index, :]
            if len(tmp_df.LAYER.unique()) == 4:
                comb.append(tmp_df)  # comb[] contains combinations of data
        tot_Hits = 4

    # saving ORBIT_CNT
    orbit = np.array(df["ORBIT_CNT"])[0]

    # saving SL
    sl = np.array(df["SL"])[0]

    flag = True

    for data in comb:
        X = np.array(pd.concat([data["X_RIGHT_GLOB"], data["X_LEFT_GLOB"]]))
        Y = np.array(pd.concat([data["WIRE_Z_GLOB"], data["WIRE_Z_GLOB"]]))
        for indexes_comb in list(combinations(range(len(X)), tot_Hits)):
            new_X = []
            new_Y = []
            for i in indexes_comb:
                new_X.append(X[i])
                new_Y.append(Y[i])

            if len(np.unique(new_Y)) == tot_Hits:
                regr_tuple = linear_reg(new_X, new_Y)
                if flag:
                    min_lambda = abs(regr_tuple["chisq_comp"])
                    xdata = new_X
                    ydata = new_Y
                    res_dict = regr_tuple
                    flag = False
                    best_comb = indexes_comb
                    best_data = data
                elif abs(regr_tuple["chisq_comp"]) < min_lambda:
                    best_comb = indexes_comb
                    min_lambda = abs(regr_tuple["chisq_comp"])
                    xdata = new_X
                    ydata = new_Y
                    res_dict = regr_tuple
                    best_data = data

    big_df = pd.concat([best_data, best_data], axis=0, ignore_index=True)
    reco_df = big_df.loc[best_comb, :]
    reco_df["m"] = np.full(len(reco_df), res_dict["m"])
    reco_df["q"] = np.full(len(reco_df), res_dict["q"])
    reco_df["X"] = xdata
    res_dict["ORBIT_CNT"] = orbit
    res_dict["SL"] = sl

    return res_dict, xdata, ydata, reco_df


### Comparison

In [12]:
n_iter  = 5
events  = getEvents_o(DATA_FNAME, cfg, 0)
df_E    = events[5]
chamber = [df_E[df_E["SL"] == i] for i in range(4)]
df      = chamber[1]
print("Chosen df eligible for computing?", len(pd.unique(df.LAYER)) >=3 )

tracemalloc.start()

t_o       = timeit('compute_o(df)', number = n_iter, globals=globals()) / n_iter
_, peak_o = tracemalloc.get_traced_memory()

tracemalloc.reset_peak()

t_no       = timeit('compute_no(df)', number = n_iter, globals=globals()) / n_iter
_, peak_no = tracemalloc.get_traced_memory()

tracemalloc.stop()

print("Not optimized:\nTime {0} s, Memory {1} MB\nOptimized:\nTime {2} s, Memory {3} MB".format(t_no, peak_no/10**6, t_o, peak_o/10**6) )

Valid hits: 54952
Chosen df eligible for computing? True
Not optimized:
Time 0.04058319420000771 s, Memory 0.091308 MB
Optimized:
Time 0.04070788639946841 s, Memory 0.082303 MB


## Multiprocessing

### Raw Code

In [13]:

# *****************************************************************************
# COMPUTE EVENT
# *****************************************************************************

def computeEvent(df_E):

    chamber = [df_E[df_E["SL"] == i] for i in range(4)]
    event_reco_df = pd.DataFrame()

    for df in chamber:
        if len(pd.unique(df.LAYER)) < 3:
            continue
        
        chamber_reco_df = compute_o(df)
        event_reco_df = pd.concat(
            [event_reco_df, chamber_reco_df], axis=0, ignore_index=True
        )

    if len(event_reco_df)==0:
        return None

    return event_reco_df

def getRecoResults(events):
    resultsDf = []

    for df_E in events:
        event_reco_df = computeEvent(df_E)
        if event_reco_df is None:
            continue
        if len(event_reco_df)==0:
            continue
        resultsDf.append(event_reco_df)

    return resultsDf

# *****************************************************************************
# MULTIPROCESSING
# *****************************************************************************
from multiprocessing import Pool, cpu_count

def getRecoResults_mp(events):
    
    pool = Pool(processes=cpu_count()-2)
    
    result = pool.map_async(computeEvent, events)
    resultsDf = result.get()
    pool.close()
    pool.join()
    resultsDf = [x for x in resultsDf if x is not None]

    return resultsDf


### Comparison

In [14]:
n_iter  = 1
#events  = getEvents_o(DATA_FNAME, cfg, 0)

tracemalloc.start()

t_o       = timeit('getRecoResults_mp(events)', number = n_iter, globals=globals()) / n_iter
_, peak_o = tracemalloc.get_traced_memory()

tracemalloc.reset_peak()

t_no       = timeit('getRecoResults(events)', number = n_iter, globals=globals()) / n_iter
_, peak_no = tracemalloc.get_traced_memory()

tracemalloc.stop()

print("Not optimized:\nTime {0} s, Memory {1} MB\nOptimized:\nTime {2} s, Memory {3} MB".format(t_no, peak_no/10**6, t_o, peak_o/10**6) )

/home/alberto/anaconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/alberto/anaconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/alberto/anaconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/alberto/anaconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/alberto/anaconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covarian

Not optimized:
Time 436.2627538680026 s, Memory 34.993542 MB
Optimized:
Time 157.51661917399906 s, Memory 24.963305 MB
